#### Q4 - 三個粉絲頁個別的「貼文數量」、「所有貼文中有被 share 的比例」、「所有貼文中有包含 Friend Tag 的比例」

想使用圓餅圖來呈現

In [1]:
%matplotlib inline
import json;
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
import seaborn as sb
sb.set_style("whitegrid")
import pyes
es_address='127.0.0.1:9200'
conn = pyes.es.ES(es_address)

## 資料集當中有 3 種 Tag:

1. **message_tags**: Profiles tagged in message. This is an object with a unique key for each tag in the message.
2. **story_tags**: Deprecated field, same as message_tags.
3. **with_tags**: Profiles tagged as being 'with' the publisher of the post.


## 而其中:

1. **message**: The status message in the post.
2. **story**: Text from stories not intentionally generated by users, such as those generated when two people become friends, or when someone else posts on the person's wall.

# 故採用 message_tags 為標準!

In [90]:
def Posts(fid):
    q = pyes.query.BoolQuery()

    tq = pyes.query.TermQuery(field="fid", value=fid)
    q.add_must(tq)

    Presult = conn.search(query=q , indices='facebook_nested' , doc_types='post') 
    return len(Presult)

def Shares(fid):
    ShareCounter = 0
    q = pyes.query.BoolQuery()

    tq = pyes.query.TermQuery(field="fid", value=fid)
    q.add_must(tq)

    Sresult = conn.search(query=q , indices='facebook_nested' , doc_types='post') 
    
    for x in range(len(Sresult)):
        if 'shares' in Sresult[x]: # 如果沒有 shares 這個 key，代表沒有人分享這則貼文。
            ShareCounter = ShareCounter + 1
    return ShareCounter

def Tags(fid):
    MessageTagCounter = 0
    
    q = pyes.query.BoolQuery()

    tq = pyes.query.TermQuery(field="fid", value=fid)
    q.add_must(tq)
    
    Tresult = conn.search(query=q , indices='facebook_nested' , doc_types='post') 
    for x in range(len(Tresult)):
        if 'story_tags' in Tresult[x]:
            #print Tresult[x]['message']
            MessageTagCounter = MessageTagCounter + 1
    return MessageTagCounter
    
def ShowResult(fid):
    resultBuffer = [Posts(fid), Shares(fid), Tags(fid)]
    print "The page id", fid, "has:"
    print "All posts count:", resultBuffer[0]
    print "Posts including share:", resultBuffer[1], "(", resultBuffer[1]*100/resultBuffer[0], "% )"
    print "Posts including tags:", resultBuffer[2], "(", resultBuffer[2]*100/resultBuffer[0], "% )\n"

ShowResult(156435698616)
ShowResult(134004310003557)
ShowResult(170290829685289)

The page id 156435698616 has:
All posts count: 2175
Posts including share: 1545 ( 71 % )
Posts including tags: 444 ( 20 % )

The page id 134004310003557 has:
All posts count: 1497
Posts including share: 1467 ( 97 % )
Posts including tags: 234 ( 15 % )

The page id 170290829685289 has:
All posts count: 2312
Posts including share: 2203 ( 95 % )
Posts including tags: 324 ( 14 % )



# 找一則有 Tag 的 comment
**根本看不出有沒有 Tag 朋友**

In [91]:
def Posts(fid):
    counter = 0
    qp = pyes.query.BoolQuery()

    tqp = pyes.query.TermQuery(field="fid", value=fid)
    qp.add_must(tqp)

    Presult = conn.search(query=qp , indices='facebook_nested' , doc_types='post') 

    for x in range(500): #len(Presult)
        qc = pyes.query.BoolQuery()
        tqc = pyes.query.TermQuery(field='message', value='yang')
        qc.add_must(tqc)

        Cresult = conn.search(query=qc , indices='facebook_nested' , doc_types='comment') 
        
        #for y in range(len(Cresult)):
            #if 'message' in Cresult[y]:
                #print Cresult[y]['message']
    print Cresult[5]

Posts(156435698616)

{u'from': {u'name': u'Becky Yang', u'id': u'10208315065822125'}, u'fid': u'76625396025', u'fid_pid': u'76625396025_10153415200106026', u'timestamp': u'2016-03-27T01:55:44+0800', u'@timestamp': u'2016-03-30T02:08:03.241Z', u'like_count': 1, u'created_at_linux_timestamp': 1448428775000L, u'can_remove': u'false', u'fname': u'CDC', u'created_time': u'2015-11-25T13:19:35+0800', u'message': u'Jessica Yang Y.p. Yang', u'@version': u'1', u'id': u'10153415200106026_10153415798626026', u'user_likes': u'false'}


# 用 ElasticSearch 也看不出所以然

{
"_index": "facebook_nested",
"_type": "comment",
"_id": "1029323543806874_1030283920377503",
"_version": 1,
"_score": null,
"_source": {
"fid": "324273577645211",
"created_time": "2015-11-04T16:59:58+0800",
"fname": "KFC",
"can_remove": "false",
"like_count": 0,
"user_likes": "false",
"from": {
"name": "芒果遊戲工作室",
"id": "424699220919366",
"category": "Sports/Recreation/Activities"
},
"id": "1029323543806874_1030283920377503",
"fid_pid": "324273577645211_1029323543806874",
"created_at_linux_timestamp": 1446627598000,
"message": "王榮典 詹喬客 方群 楊慶之 Ki Wi Chang 加班辛苦了! 讓肯德基星馬肉骨茶紙包雞來幫我們暖心又暖胃! #KFC星馬肉骨茶紙包雞",
"timestamp": "2016-03-26T11:54:14+0800",
"@version": "1",
"@timestamp": "2016-03-30T02:14:57.730Z"
},
"sort": [
"activities"
,
"facebook_nested"
]
}

In [117]:
def Posts(fid):
    qc = pyes.query.BoolQuery()
    tqc = pyes.query.TermQuery(field='fid_pid', value="324273577645211_1029323543806874")
    qc.add_must(tqc)

    Cresult = conn.search(query=qc , indices='facebook_nested' , doc_types='comment') 

    #for y in range(len(Cresult)):
        #if 'message' in Cresult[y]:
            #print Cresult[y]['message']
    for y in range(10):
        print Cresult[y]
        #if 'message_tags' in Cresult[y]:
            #print Cresult[y]['message_tags']

Posts(156435698616)

{u'from': {u'name': u'\u5468\u4efb\u54f2', u'id': u'532389616924943'}, u'fid': u'324273577645211', u'fid_pid': u'324273577645211_1029323543806874', u'timestamp': u'2016-03-26T11:54:14+0800', u'@timestamp': u'2016-03-30T02:03:11.278Z', u'like_count': 0, u'created_at_linux_timestamp': 1446626526000L, u'can_remove': u'false', u'fname': u'KFC', u'created_time': u'2015-11-04T16:42:06+0800', u'message': u'Chen Piter Marry Cho Sqwzx Erdfu Fnge Min Vemary Lo  \u52a0\u73ed\u8f9b\u82e6\u4e86! \u8b93\u80af\u5fb7\u57fa\u661f\u99ac\u8089\u9aa8\u8336\u7d19\u5305\u96de\u4f86\u5e6b\u6211\u5011\u6696\u5fc3\u53c8\u6696\u80c3!\n#KFC\u661f\u99ac\u8089\u9aa8\u8336\u7d19\u5305\u96de', u'@version': u'1', u'id': u'1029323543806874_1030278363711392', u'user_likes': u'false'}
{u'from': {u'name': u'Marry Cho', u'id': u'445344329004714'}, u'fid': u'324273577645211', u'fid_pid': u'324273577645211_1029323543806874', u'timestamp': u'2016-03-26T11:54:19+0800', u'@timestamp': u'2016-03-30T02:03:11.278Z', u'like_count'